In [ ]:
import bs4 as bs
import requests
import urllib.request
import pandas as pd

In [ ]:
url = 'https://www.worldometers.info/coronavirus/'

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}

r = requests.get(url, headers=header)

# Read text result into Pandas
dfs = pd.read_html(r.text)
# Look for the first table
df = dfs[0]
# Fill missing values with 0
df = df.fillna(0)

df.to_csv('./Data/world_info.csv')

In [ ]:
homepage_soup = bs.BeautifulSoup(r.text, 'lxml')
# Using the table on the main page, only the countries with links (<a> tags) have detailed historical data.
country_elements = homepage_soup.select('table[id="main_table_countries_today"] > tbody > tr > td > a')
# Start with an empty list
countries_with_detailed_data = []
# Iterate through each element, and add the contents (country name) to a list
for individual_element in country_elements:
    countries_with_detailed_data.append(individual_element.contents[0])
# Table is sorted by total cases. Sort alphabetically instead
countries_with_detailed_data.sort()

print("Can get detailed data for %s" % countries_with_detailed_data)

In [ ]:
country = 'Canada'
url = 'https://www.worldometers.info/coronavirus/country/' + country
r = requests.get(url, headers=header).text

soup = bs.BeautifulSoup(r, 'lxml')

cases = soup.find('div', class_='col-md-12').get_text(strip=True)
cases = cases.split(',', 100)
del cases[0:8]

scripts = soup.select('script[type="text/javascript"]')
for individual_script in scripts:
    individual_script = individual_script.get_text(strip=True)
    if("coronavirus-deaths-linear" in individual_script):
        deaths = individual_script
deaths = deaths.split(',', 100)

#print(cases)
#print(deaths)

In [ ]:
case_dates = cases[0:38]

for i in range(0,len(case_dates)):
    case_dates[i] = case_dates[i][:-1]
    case_dates[i] = case_dates[i][1:]
    
case_dates[len(case_dates)-1] = case_dates[len(case_dates)-1][:-10]
case_dates[0] = case_dates[0][44:]

#print(case_dates)
#len(case_dates)

In [ ]:
case_data = cases[46:84]

case_data[len(case_data)-1] = case_data[len(case_data)-1][:-11]    
case_data[0] = case_data[0][20:]

for i in range(0,len(case_data)):
    case_data[i] = int(case_data[i])

#print(case_data)
#len(case_data)

In [ ]:
death_data = deaths[50:88]

death_data[len(death_data)-1] = death_data[len(death_data)-1][:-15]    
death_data[0] = death_data[0][24:]

for i in range(0,len(death_data)):
    death_data[i] = int(death_data[i])

#print(death_data)
#len(death_data)

In [ ]:
df_country = pd.DataFrame(
    {'Dates': case_dates,
     'Cases': case_data,
     'Deaths': death_data
    })

df_country = df_country.fillna(0)

file_name = './Data/Countries/' + country + '.csv'
df_country.to_csv(file_name)
